In [1]:
%matplotlib inline
%run ../../path_import.py
import numpy as np
import matplotlib.pyplot as plt
from plots import *
from utils import *
from training import *
from model_mu_reparam import *
import time
import probtorch
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.1.0 cuda: True


In [2]:
Data = torch.from_numpy(np.load('../rings_fixed_radius/obs.npy')).float()
## Data Parameters
N = 300
K = 3
D = 2
FIXED_RADIUS = 1.5
## Model Parameters
MCMC_SIZE = 20
SAMPLE_SIZE = 10
NUM_HIDDEN_GLOBAL = 8
NUM_HIDDEN_LOCAL = 64
STAT_SIZE = 8
NUM_LATENTS =  D
## Training Parameters
BATCH_SIZE = 20
NUM_EPOCHS = 500
LEARNING_RATE =  1e-4
CUDA = torch.cuda.is_available()
PATH = 'ag-pr-%dsteps-%dsamples' % (MCMC_SIZE, SAMPLE_SIZE)
DEVICE = torch.device('cuda:1')

In [3]:
enc_mu = Enc_mu(K, D, num_hidden=NUM_HIDDEN_GLOBAL, num_stats=STAT_SIZE, CUDA=CUDA, device=DEVICE)
enc_z = Gibbs_z(K, CUDA, DEVICE)
obs_rad = torch.ones(1) * FIXED_RADIUS
noise_sigma = torch.ones(1) * 0.05
if CUDA:
    enc_mu.cuda().to(DEVICE)
    obs_rad = obs_rad.cuda().to(DEVICE)
    noise_sigma =noise_sigma.cuda().to(DEVICE)
optimizer =  torch.optim.Adam(list(enc_mu.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))

In [ ]:
RESAMPLE=True
DETACH=True
train_mu(Eubo_cfz_pr_z, enc_mu, enc_z, optimizer, Data, obs_rad, noise_sigma, K, NUM_EPOCHS, MCMC_SIZE, SAMPLE_SIZE, BATCH_SIZE, PATH, CUDA, DEVICE, RESAMPLE=RESAMPLE, DETACH=DETACH)

epoch=0, SymKL=3776644.578, EUBO=-625084.712, ELBO=-4560096.430, ESS=1.054 (45s)
epoch=1, SymKL=3320021.325, EUBO=-573019.798, ELBO=-4033113.120, ESS=1.047 (46s)
epoch=2, SymKL=2912934.993, EUBO=-522580.194, ELBO=-3555132.748, ESS=1.043 (46s)
epoch=3, SymKL=2551542.535, EUBO=-474093.484, ELBO=-3131158.695, ESS=1.036 (46s)
epoch=4, SymKL=2243908.462, EUBO=-424241.150, ELBO=-2760522.150, ESS=1.032 (46s)
epoch=5, SymKL=1967168.959, EUBO=-381070.916, ELBO=-2430194.626, ESS=1.028 (46s)
epoch=6, SymKL=1730179.028, EUBO=-337177.480, ELBO=-2139900.071, ESS=1.024 (45s)
epoch=7, SymKL=1519892.826, EUBO=-302968.920, ELBO=-1887525.841, ESS=1.019 (44s)
epoch=8, SymKL=1339582.209, EUBO=-265965.411, ELBO=-1664174.240, ESS=1.016 (39s)
epoch=9, SymKL=1173585.997, EUBO=-235120.892, ELBO=-1462563.574, ESS=1.013 (45s)
epoch=10, SymKL=1025993.047, EUBO=-207052.352, ELBO=-1282623.288, ESS=1.012 (46s)
epoch=11, SymKL=895584.726, EUBO=-183562.259, ELBO=-1125692.151, ESS=1.011 (45s)
epoch=12, SymKL=777792.007,

In [ ]:
torch.save(enc_mu.state_dict(), "../weights/enc-mu-%s" % PATH)
# torch.save(enc_z.state_dict(), "../weights/enc-z-%s" % PATH)

In [ ]:
BATCH_SIZE_TEST = 50
obs, q_mu, q_z, symkls_test, eubos_test, elbos_test = test(Eubo_cfz_init_eta, enc_mu, enc_z, Data, obs_rad, noise_sigma, K, 10, SAMPLE_SIZE, BATCH_SIZE_TEST, CUDA, DEVICE, RESAMPLE, DETACH=DETACH)
%time plot_samples(obs, q_mu, q_z, K, PATH)

In [ ]:
incremental_gap = symkls_test.cpu().data.numpy()[1:]
M = incremental_gap.shape[0]
overall_gap = np.zeros(M)
for m in range(M):
    overall_gap[m] = incremental_gap[:m+1].sum()

In [ ]:
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(111)
plt.yscale("log")
ax.plot(incremental_gap, label="incremental gap")
ax.plot(overall_gap, label='overall gap')
ax.legend(fontsize=14)
ax.set_xlabel('Steps')